In [24]:
cd C:\Users\LukaszMigas\Dropbox (The University of Manchester)\Barran_group_folder\SPREADSHEETS

C:\Users\LukaszMigas\Dropbox (The University of Manchester)\Barran_group_folder\SPREADSHEETS


In [25]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource,Column 
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
import seaborn as sns
import random
output_notebook()
%matplotlib inline
from IPython.display import HTML

def randomColorGenerator():
    colorGen = ''.join([random.choice('0123456789ABCDEF') for x in range(6)])
    return ''.join(["#",colorGen])

Loading BokehJS ...

In [43]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Interactive tools:
#### <font color='green'>Hover</font>  - hover over any data point and it will bring detailed information about each curve (index point, CCS value, CCSD intensity and charge state)
#### <font color='green'>Zoom</font> - drag on the plot to zoom on specific region of interest
#### <font color='green'>Legend</font> - click on legend marker to select/deselect datasets 

In [4]:
# To do...
# Add possibility to normalize data to 1
# Add possibility to fill under the curve

In [41]:
filename = "Combined CCSD data for EU Herceptin for onoline notebook - LM.csv"
def dataLoad(filename=None):
    """ Function to load data from CSV file """
    # Import data
    data = pd.read_csv(filename, delimiter=',')
    # Extract CCS data
    ccsData = data.CCS
    #intGlobal = data['global']
    intCharge = data.loc[:, data.columns.str.startswith('z')]
    dataOut = [ccsData, intCharge]
    return dataOut, data

def prepareBokehTool():
    """ Function to prepare a tool for use with the Bokeh plot"""
    hover = HoverTool(tooltips=[("Point #:", "$index"),
                               ("CCS (nm²):", "$x{int} nm²"),
                               ("Normalized intensity:", "$y{0.00}"),
                               ("Charge state:", "@chargeState"),
                               ("File info:", "@fileInfo")])
    TOOLS = [hover,"box_zoom,resize,pan,wheel_zoom,reset,save"]
    return hover, TOOLS


def makeBokehFigure(dataIn=None, tool=None, plotWidth=800, plotHeight=750, xRange=(500,3000),
                   yRange=(0,1.05), title=None, activeTool="box_zoom", tabbed=False):
    
    t = figure(tools=tool, plot_width=plotWidth, plot_height=plotHeight, x_range=xRange
              , y_range=yRange, title=title, active_drag=activeTool)
    i = 0 
    dataSource = []
    ccsData, intCharge = dataOut
    for key in intCharge:
        chargeState = ''.join([key[1:3],"+"]) # assumes the first two numbers are charge state
        fileInfo = ''.join([key[4::]]) # assumes there is info in the name 
        legendKey = ''.join([chargeState,' (',fileInfo,')']) # makes a nicer label
        source = ColumnDataSource(data=dict(x=ccsData, 
                                            y=intCharge[key], 
                                            xlabel=([legendKey]*(len(ccsData))),
                                            chargeState=([chargeState]*(len(ccsData))),
                                            fileInfo=([fileInfo]*(len(ccsData)))))
        dataSource.append(source)
        t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
              source=dataSource[i]) 
        i=i+1
    # If there was global IMS data, it could be added here
    #source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
    #dataSource.append(source)
    #t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])       
    
    t.title.text_font_size = "16pt"
    t.xaxis.major_label_text_font_size = "12pt"
    t.xaxis.axis_label = 'Collision Cross Section (nm²)'
    t.xaxis.axis_label_text_font_size = "16pt"
    t.yaxis.major_label_text_font_size = "12pt"
    t.yaxis.axis_label = 'Normalized Intensity'
    t.yaxis.axis_label_text_font_size = "16pt"
    t.legend.click_policy="hide"
    if tabbed == True:
        tab = Panel(child=t, title=title)
        return tab
    else:
        show(t)
        return None

# <font color='black'>Antibody comparison</font> 

In [44]:
plotHeight = 650
title = 'Antibody'
dataOut, data = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(50,90),
                yRange=(0,110), title=title, activeTool="box_zoom")